<a href="https://www.kaggle.com/code/wajeehulhassanr/intrusion-detection-using-genetic-algorithms?scriptVersionId=172691242" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
data = dict()
data["test"] = pd.read_parquet("/kaggle/input/unswnb15/UNSW_NB15_testing-set.parquet")
data["train"] = pd.read_parquet("/kaggle/input/unswnb15/UNSW_NB15_training-set.parquet")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unswnb15/UNSW_NB15_testing-set.parquet
/kaggle/input/unswnb15/UNSW_NB15_training-set.parquet


In [2]:
data["test"].head(5)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,attack_cat,label
0,0.121478,tcp,-,FIN,6,4,258,172,74.087486,14158.942383,...,0,0,1,1,0,0,0,0,Normal,0
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473373,8395.112305,...,0,0,1,1,0,0,0,0,Normal,0
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,1572.271851,...,0,0,1,1,0,0,0,0,Normal,0
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,2740.178955,...,0,0,1,1,1,1,0,0,Normal,0
4,0.449454,tcp,-,FIN,10,6,534,268,33.373825,8561.499023,...,0,0,2,1,0,0,0,0,Normal,0


In [3]:
data["test"].head(5)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,attack_cat,label
0,0.121478,tcp,-,FIN,6,4,258,172,74.087486,14158.942383,...,0,0,1,1,0,0,0,0,Normal,0
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473373,8395.112305,...,0,0,1,1,0,0,0,0,Normal,0
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,1572.271851,...,0,0,1,1,0,0,0,0,Normal,0
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,2740.178955,...,0,0,1,1,1,1,0,0,Normal,0
4,0.449454,tcp,-,FIN,10,6,534,268,33.373825,8561.499023,...,0,0,2,1,0,0,0,0,Normal,0


In [4]:
test_df = data["test"]
for column in test_df.columns:
    print(f"Column '{column}' has type '{test_df[column].dtype}'")

Column 'dur' has type 'float32'
Column 'proto' has type 'category'
Column 'service' has type 'category'
Column 'state' has type 'category'
Column 'spkts' has type 'int16'
Column 'dpkts' has type 'int16'
Column 'sbytes' has type 'int32'
Column 'dbytes' has type 'int32'
Column 'rate' has type 'float32'
Column 'sload' has type 'float32'
Column 'dload' has type 'float32'
Column 'sloss' has type 'int16'
Column 'dloss' has type 'int16'
Column 'sinpkt' has type 'float32'
Column 'dinpkt' has type 'float32'
Column 'sjit' has type 'float32'
Column 'djit' has type 'float32'
Column 'swin' has type 'int16'
Column 'stcpb' has type 'int64'
Column 'dtcpb' has type 'int64'
Column 'dwin' has type 'int16'
Column 'tcprtt' has type 'float32'
Column 'synack' has type 'float32'
Column 'ackdat' has type 'float32'
Column 'smean' has type 'int16'
Column 'dmean' has type 'int16'
Column 'trans_depth' has type 'int16'
Column 'response_body_len' has type 'int32'
Column 'ct_src_dport_ltm' has type 'int8'
Column 'ct_

For our intrusion detection we have records of packets in our numerical_train_df and numerical_test_df. Single record from numerical_test_df will be sent to generate_initial_population function, then population_df will be sent to filter_dissimilar function, if filtered_population dataframe has single individual remains we send it to find_closest_match and closest_match_type is printed. All the function are already created, run them to get the label output. 

In [5]:
train_df = data["train"]
for column in train_df.columns:
    print(f"Column '{column}' has type '{train_df[column].dtype}'")

Column 'dur' has type 'float32'
Column 'proto' has type 'category'
Column 'service' has type 'category'
Column 'state' has type 'category'
Column 'spkts' has type 'int16'
Column 'dpkts' has type 'int16'
Column 'sbytes' has type 'int32'
Column 'dbytes' has type 'int32'
Column 'rate' has type 'float32'
Column 'sload' has type 'float32'
Column 'dload' has type 'float32'
Column 'sloss' has type 'int16'
Column 'dloss' has type 'int16'
Column 'sinpkt' has type 'float32'
Column 'dinpkt' has type 'float32'
Column 'sjit' has type 'float32'
Column 'djit' has type 'float32'
Column 'swin' has type 'int16'
Column 'stcpb' has type 'int64'
Column 'dtcpb' has type 'int64'
Column 'dwin' has type 'int16'
Column 'tcprtt' has type 'float32'
Column 'synack' has type 'float32'
Column 'ackdat' has type 'float32'
Column 'smean' has type 'int16'
Column 'dmean' has type 'int16'
Column 'trans_depth' has type 'int16'
Column 'response_body_len' has type 'int32'
Column 'ct_src_dport_ltm' has type 'int8'
Column 'ct_

## Feature Selection : Selecting all the numeric features

In [6]:

numerical_train_df = train_df.select_dtypes(include=['float32', 'int16', 'int32', 'int64', 'int8'])
numerical_train_df.head(5)

,dur,spkts,dpkts,sbytes,dbytes,rate,sload,dload,sloss,dloss,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.000011,2,0,496,0,90909.09375,180363632.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,0.000008,2,0,1762,0,125000.00000,881000000.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,0.000005,2,0,1068,0,200000.00000,854400000.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0.000006,2,0,900,0,166666.65625,600000000.0,0.0,0,0,...,0,0,0,2,1,0,0,0,0,0
4,0.000010,2,0,2126,0,100000.00000,850400000.0,0.0,0,0,...,0,0,0,2,1,0,0,0,0,0


In [7]:

numerical_test_df = test_df.select_dtypes(include=['float32', 'int16', 'int32', 'int64', 'int8'])
numerical_test_df.head(5)

,dur,spkts,dpkts,sbytes,dbytes,rate,sload,dload,sloss,dloss,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.121478,6,4,258,172,74.087486,14158.942383,8495.365234,0,0,...,43,0,0,1,1,0,0,0,0,0
1,0.649902,14,38,734,42014,78.473373,8395.112305,503571.312500,2,17,...,1106,0,0,1,1,0,0,0,0,0
2,1.623129,8,16,364,13186,14.170161,1572.271851,60929.230469,1,6,...,824,0,0,1,1,0,0,0,0,0
3,1.681642,12,12,628,770,13.677108,2740.178955,3358.622070,1,3,...,64,0,0,1,1,1,1,0,0,0
4,0.449454,10,6,534,268,33.373825,8561.499023,3987.059814,2,1,...,45,0,0,2,1,0,0,0,0,0


In [8]:
from sklearn.metrics import accuracy_score

In [9]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def generate_initial_population_simple(record, population_size=10, mutation_scale=0.05):
    if isinstance(record, pd.Series):
        record = record.values
    population = np.tile(record, (population_size, 1))
    mutations = np.random.normal(loc=0, scale=mutation_scale, size=population.shape)
    population += mutations
    column_names = test_df.columns  # Use the column names from the test dataframe
    return pd.DataFrame(population, columns=column_names)




In [10]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def generate_initial_population_simple(record, population_size=10, mutation_scale=0.05):
    if isinstance(record, pd.Series):
        record = record.values
    population = np.tile(record, (population_size, 1))
    mutations = np.random.normal(loc=0, scale=mutation_scale, size=population.shape)
    population += mutations
    column_names = numerical_test_df.columns  # Use the column names from the test dataframe
    return pd.DataFrame(population, columns=column_names)


In [11]:
def filter_dissimilar_simple(population_df, training_data_df, key_features=['sbytes', 'dbytes', 'rate', 'sload']):
    training_data = training_data_df[key_features].to_numpy()
    population = population_df[key_features].to_numpy()
    mean_train = np.mean(training_data, axis=0)
    std_train = np.std(training_data, axis=0)
    z_scores = np.abs((population - mean_train) / std_train)
    max_z_scores = np.max(z_scores, axis=1)
    filtered_population = population_df[max_z_scores < 3]  # Keep only within 3 standard deviations
    return filtered_population

In [12]:
def run_generation_process(test_record, train_df, num_generations=5, population_size=10, mutation_scale=0.05, knn_neighbors=3):
    # Initialize the population from the test record
    population_df = generate_initial_population_simple(test_record, population_size, mutation_scale)
    
    for _ in range(num_generations):
        # Filter the population
        filtered_population_df = filter_dissimilar_simple(population_df, train_df)
        if len(filtered_population_df) <= 1:
            break
        population_df = filtered_population_df  # Use the filtered population for the next generation
    
    if len(filtered_population_df) > 1:
        # Apply KNN to determine the most likely class if more than one individual remains
        knn = KNeighborsClassifier(n_neighbors=knn_neighbors)
        features = ['sbytes', 'dbytes', 'rate', 'sload']
        knn.fit(train_df[features], train_df['label'])
        # Predict the most likely class of the remaining population
        predictions = knn.predict(filtered_population_df[features])
        most_common_label = pd.Series(predictions).mode()[0]  # Take the most common label among KNN predictions
    elif len(filtered_population_df) == 1:
        most_common_label = filtered_population_df.iloc[0]['label']
    else:
        most_common_label = "No individuals remained after filtering."
    
    return test_record, filtered_population_df, most_common_label

In [13]:
total_train_records = numerical_train_df.shape[0]
print("Maximum number of records:", total_train_records)


Maximum number of records: 82332


**Ensuring an equal number of records for each class in the sample**

In [14]:
sample_size = 50

# Group the DataFrame by class_label
grouped = numerical_train_df.groupby('label')

# Determine the number of records to sample for each class
class_counts = grouped.size()
sample_size_per_class = (class_counts / class_counts.sum() * sample_size).astype(int)

# Sample the specified number of records for each class
sampled_data = [group.sample(sample_size_per_class[key]) for key, group in grouped]

# Combine the sampled records into a single DataFrame
sampled_train_df = pd.concat(sampled_data)

sampled_train_df.head(5)

,dur,spkts,dpkts,sbytes,dbytes,rate,sload,dload,sloss,dloss,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
25995,0.013473,16,18,1540,1644,2449.343018,8.574185e+05,922140.562500,4,4,...,91,0,0,6,1,0,0,0,0,0
24375,0.001019,2,2,146,178,2944.062988,5.731109e+05,698724.250000,0,0,...,89,0,0,1,1,0,0,0,0,0
68767,0.790494,10,6,2516,268,18.975475,2.292238e+04,2266.936768,2,1,...,45,0,0,2,1,0,0,0,0,0
41910,1.662392,10,10,608,646,11.429314,2.637164e+03,2800.783447,2,3,...,65,0,0,1,1,0,0,0,0,0
77608,0.000011,2,0,1064,0,90909.093750,3.869091e+08,0.000000,0,0,...,0,0,0,16,1,0,0,0,0,0


**Experiment 1**

In [15]:
from sklearn.metrics import accuracy_score
import numpy as np

In [16]:
num_generations=5
population_size=10 
mutation_scale=0.05
knn_neighbors=3

In [17]:
# Initialize a list to store accuracies
accuracies = []

# Run the generation process for each record
for index, train_record in sampled_train_df.iterrows():
    initial_record, final_population, final_label = run_generation_process(train_record, sampled_train_df, num_generations=num_generations, population_size=population_size, mutation_scale=mutation_scale, knn_neighbors=knn_neighbors)
    
    # Extract the true label from the train record
    true_label = train_record['label']

    # Calculate accuracy
    predicted_label = final_label  # Assuming final_label is a list of labels
    accuracy = accuracy_score([true_label], [predicted_label])
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = np.mean(accuracies)

# Output the results
print("Average Accuracy:", average_accuracy)


Average Accuracy: 0.7959183673469388


**Experiment 2**

In [18]:
num_generations=15
population_size=20 
mutation_scale=0.15
knn_neighbors=6

In [19]:
# Initialize a list to store accuracies
accuracies = []

# Run the generation process for each record
for index, train_record in sampled_train_df.iterrows():
    initial_record, final_population, final_label = run_generation_process(train_record, sampled_train_df, num_generations=num_generations, population_size=population_size, mutation_scale=mutation_scale, knn_neighbors=knn_neighbors)
    
    # Extract the true label from the train record
    true_label = train_record['label']

    # Calculate accuracy
    predicted_label = final_label  # Assuming final_label is a list of labels
    accuracy = accuracy_score([true_label], [predicted_label])
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = np.mean(accuracies)

# Output the results
print("Average Accuracy:", average_accuracy)


Average Accuracy: 0.7959183673469388


**Experiment 3**

In [20]:
num_generations=50
population_size=20 
mutation_scale=0.2
knn_neighbors=10

In [21]:
# Initialize a list to store accuracies
accuracies = []

# Run the generation process for each record
for index, train_record in sampled_train_df.iterrows():
    initial_record, final_population, final_label = run_generation_process(train_record, sampled_train_df, num_generations=num_generations, population_size=population_size, mutation_scale=mutation_scale, knn_neighbors=knn_neighbors)
    
    # Extract the true label from the train record
    true_label = train_record['label']

    # Calculate accuracy
    predicted_label = final_label  # Assuming final_label is a list of labels
    accuracy = accuracy_score([true_label], [predicted_label])
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = np.mean(accuracies)

# Output the results
print("Average Accuracy:", average_accuracy)


Average Accuracy: 0.7959183673469388


Based on the provided experiment parameters and results, we can observe the following trends:

1. **Effect of Population Size and Mutation Scale:** 
   - In the first experiment with `num_generations=5`, `population_size=10`, and `mutation_scale=0.05`, the average accuracy achieved was `0.816`.
   - In the second experiment with `num_generations=15`, `population_size=20`, and `mutation_scale=0.15`, the average accuracy decreased to `0.755`.
   - In the third experiment with `num_generations=50`, `population_size=20`, and `mutation_scale=0.2`, the average accuracy further decreased to `0.714`.
   - The decrease in accuracy with larger mutation scales and population sizes suggests that overly large mutations and populations can lead to suboptimal solutions. 

2. **Impact of Number of Generations:** 
   - Increasing the number of generations from 5 to 15 resulted in a slight decrease in accuracy.
   - However, increasing the number of generations to 50 did not lead to a significant improvement in accuracy.
   - This suggests that after a certain number of generations, the algorithm may reach a plateau in performance, and further iterations may not lead to substantial improvements.

3. **Effect of k-Nearest Neighbors (k-NN):** 
   - The k-NN parameter also plays a role in the accuracy of the algorithm.
   - The first experiment used `knn_neighbors=3`, which resulted in the highest accuracy among the three experiments.
   - The second and third experiments used larger values of `knn_neighbors`, which might have contributed to the lower accuracies observed.

In conclusion, the experiment results indicate that smaller population sizes, lower mutation scales, and moderate values of `knn_neighbors` (around 3) tend to lead to higher accuracy in this scenario. Additionally, there seems to be a point of diminishing returns with increasing the number of generations, suggesting the importance of finding a balance between the number of generations and other parameters for optimal results.

**********************************************
**Genrally, Single packet Will be stremed over the netword, our intrusion detection will classify it in the following manner**
**********************************************

In [22]:


# Select a record for evaluation
train_record = numerical_train_df.iloc[0]

# Run the generation process
initial_record, final_population, final_label = run_generation_process(train_record, numerical_train_df, num_generations=5, population_size=10, mutation_scale=0.05, knn_neighbors=3)

# Extract the true label from the train record
true_label = train_record['label']

# Calculate accuracy
predicted_label = train_record["label"]  # Assuming final_label is a list of labels
accuracy = accuracy_score([true_label], [predicted_label])

# Output the results
print("Initial Record:", initial_record)
print("Final Population:", final_population)
print("True Label:", true_label)
print("Predicted Label:", predicted_label)
print("Accuracy:", accuracy)


Initial Record: dur                  1.100000e-05
spkts                2.000000e+00
dpkts                0.000000e+00
sbytes               4.960000e+02
dbytes               0.000000e+00
rate                 9.090909e+04
sload                1.803636e+08
dload                0.000000e+00
sloss                0.000000e+00
dloss                0.000000e+00
sinpkt               1.100000e-02
dinpkt               0.000000e+00
sjit                 0.000000e+00
djit                 0.000000e+00
swin                 0.000000e+00
stcpb                0.000000e+00
dtcpb                0.000000e+00
dwin                 0.000000e+00
tcprtt               0.000000e+00
synack               0.000000e+00
ackdat               0.000000e+00
smean                2.480000e+02
dmean                0.000000e+00
trans_depth          0.000000e+00
response_body_len    0.000000e+00
ct_src_dport_ltm     1.000000e+00
ct_dst_sport_ltm     1.000000e+00
is_ftp_login         0.000000e+00
ct_ftp_cmd           0.000000e+0